In [1]:
# import dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# display multiple print results on one line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# expands .describe with more info about the data set
import pandas_profiling

# sqlalchemy dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
from sqlalchemy import inspect

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Text

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# flask 
from flask import Flask, jsonify

height has been deprecated.



In [2]:
# Use SQLAlchemy create_engine to connect to database
engine = create_engine("sqlite:///bbb.sqlite")
conn = engine.connect()

In [3]:
# use SQLAlchemy automap_base() to reflect tables into classes and save references
# called sample_names, otu_des, sample_meta, frequency
Base = automap_base()

In [4]:
# use the base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [5]:
Base.classes.keys()

['otu', 'samples', 'samples_metadata']

In [6]:
# assign classes to variables
sample_names = Base.classes.samples
otu_des = Base.classes.otu
sample_meta = Base.classes.samples_metadata

In [7]:
# create a session
# a session means that you want to do something to your data
session = Session(engine)

In [8]:
inspector = inspect(engine)

In [9]:
# get table information
print (inspector.get_table_names())

['otu', 'samples', 'samples_metadata']


In [10]:
# get col names for samples and stores in a tuple within a list
sample_name_col = inspector.get_columns('samples')

In [11]:
# prints list of sample names
sample_names_list = [item["name"] for item in sample_name_col]
del sample_names_list[0]

In [12]:
# prints list of sample_names
# sample_names_list

In [13]:
# return list of OTU descriptions
otu_results = session.query(otu_des.lowest_taxonomic_unit_found).all()

# stores otu results as a list
otu_results_list = list(np.ravel(otu_results))
# otu_results_list

In [14]:
# read Belly_Button_Biodiversity_Metadata
bbb_meta_df = pd.read_csv('Belly_Button_Biodiversity_Metadata.csv')

In [15]:
bbb_meta_df.head()
bbb_meta_df.shape

,SAMPLEID,EVENT,ETHNICITY,GENDER,AGE,WFREQ,BBTYPE,LOCATION,COUNTRY012,ZIP012,COUNTRY1319,ZIP1319,DOG,CAT,IMPSURFACE013,NPP013,MMAXTEMP013,PFC013,IMPSURFACE1319,NPP1319,MMAXTEMP1319,PFC1319
0,940,BellyButtonsScienceOnline,Caucasian,F,24.0,2.0,I,Beaufort/NC,usa,22306,usa,22306,no,no,8852.0,37.172222,54.5,NaN,1,NaN,33.990002,25.5
1,941,NaN,Caucasian/Midleastern,F,34.0,1.0,I,Chicago/IL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,943,BellyButtonsScienceOnline,Caucasian,F,49.0,1.0,I,Omaha/NE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,944,BellyButtonsScienceOnline,European,M,44.0,1.0,I,NewHaven/CT,usa,7079,usa,8822,no,yes,NaN,35.816666,16.0,NaN,0,6567.0,32.403332,28.5
4,945,BellyButtonsScienceOnline,Caucasian,F,48.0,1.0,I,Philidelphia/PA,usa,84404,usa,96025,no,no,NaN,37.783333,4.0,NaN,0,5613.0,33.634445,24.0


(153, 22)

In [16]:
# create new bbb meta_df with the following columns:
# AGE, BBTYPE, ETHNICITY, GENDER, LOCATION, SAMPLEID

bbb_meta_clean = bbb_meta_df[['AGE', 'BBTYPE', 'ETHNICITY',
                             'GENDER', 'LOCATION', 'SAMPLEID']]
bbb_meta_clean.head()
bbb_meta_clean.shape

,AGE,BBTYPE,ETHNICITY,GENDER,LOCATION,SAMPLEID
0,24.0,I,Caucasian,F,Beaufort/NC,940
1,34.0,I,Caucasian/Midleastern,F,Chicago/IL,941
2,49.0,I,Caucasian,F,Omaha/NE,943
3,44.0,I,European,M,NewHaven/CT,944
4,48.0,I,Caucasian,F,Philidelphia/PA,945


(153, 6)

# add this section to app - covers the metadata route

In [17]:
# USE FOR metadata/<sample> route - sets index to SAMPLEID
bbb_meta_clean2 = bbb_meta_clean.set_index('SAMPLEID')
bbb_meta_clean2.head()

,AGE,BBTYPE,ETHNICITY,GENDER,LOCATION
SAMPLEID,,,,,
940,24.0,I,Caucasian,F,Beaufort/NC
941,34.0,I,Caucasian/Midleastern,F,Chicago/IL
943,49.0,I,Caucasian,F,Omaha/NE
944,44.0,I,European,M,NewHaven/CT
945,48.0,I,Caucasian,F,Philidelphia/PA


In [18]:
# this will be our sample input
sample = 'BB_944'

In [19]:
# strips the BB_ prefix from the sample variable from the line above
sampleID = int(sample.replace('BB_', ""))
sampleID

944

In [20]:
# prints out the metadata for the sample input
# HERE - .loc  is locating the varibable sampleID (stripped variable 944 (sample - input var))
bbb_ID = bbb_meta_clean2.loc[sampleID] 
bbb_ID

AGE                   44
BBTYPE                 I
ETHNICITY       European
GENDER                 M
LOCATION     NewHaven/CT
Name: 944, dtype: object

In [21]:
# not sure if we need this line
bbb_ID['SAMPLEID'] = sampleID
bbb_ID

AGE                   44
BBTYPE                 I
ETHNICITY       European
GENDER                 M
LOCATION     NewHaven/CT
SAMPLEID             944
Name: 944, dtype: object

In [22]:
bbb_ID

AGE                   44
BBTYPE                 I
ETHNICITY       European
GENDER                 M
LOCATION     NewHaven/CT
SAMPLEID             944
Name: 944, dtype: object

In [23]:
# takes object and formats into a dict
bbb_ID.to_dict()

{'AGE': 44.0,
 'BBTYPE': 'I',
 'ETHNICITY': 'European',
 'GENDER': 'M',
 'LOCATION': 'NewHaven/CT',
 'SAMPLEID': 944}

In [24]:
# store object into results variable 
results = bbb_ID[["AGE","BBTYPE","ETHNICITY","GENDER","LOCATION",
    "SAMPLEID"]]
results

AGE                   44
BBTYPE                 I
ETHNICITY       European
GENDER                 M
LOCATION     NewHaven/CT
SAMPLEID             944
Name: 944, dtype: object

In [25]:
# results to dict - use this in route
meta_dict = results.to_dict()
meta_dict

{'AGE': 44.0,
 'BBTYPE': 'I',
 'ETHNICITY': 'European',
 'GENDER': 'M',
 'LOCATION': 'NewHaven/CT',
 'SAMPLEID': 944}

# END of NEW SECTION

<br>
***test section***

In [26]:
# bbb_meta_dict = bbb_meta_clean.to_dict(orient='records')

In [27]:
# bbb_meta_dict

In [28]:
# bbb_meta_json = bbb_meta_clean.to_json(orient='records')

In [29]:
# bbb_meta_json

In [30]:
# read in otu csv
otu_df1 = pd.read_csv('belly_button_biodiversity_otu_id.csv',
                     dtype=object)
# otu_df1.head()

In [31]:
otu_df1_sort = otu_df1.sort_values('otu_id', ascending=False)
# otu_df1_sort.head()

In [32]:
otu_df = otu_df1_sort[['otu_id']]
# otu_df.head()
# otu_df.shape

In [33]:
# sets otu_id in otu_df to a new list called value
# we will add this list to our dictionary - sample_dict
value = otu_df['otu_id'].tolist()

In [34]:
# this strips the quotes from value
value = list(map(int, value))

In [35]:
# sorts the SAMPLE ID column from bbb_meta_df
sample_sort = bbb_meta_df.sort_values('SAMPLEID', ascending=False)

In [36]:
# creates a descending list of SAMPLEID
sample_value = sample_sort['SAMPLEID'].tolist()

In [37]:
# strip quotes from 'otu_ids'
key1 = 'otu_ids'
key1_new = key1.strip()
print(key1_new)

otu_ids


In [38]:
# dictionary {otu_ids: [1,2,3]}
# the above is the kind of dictionary we want to make
sample_dict = {'otu_ids': value, 'sample_values': sample_value}

In [39]:
# sample_dict

In [40]:
# append value
# sample_dict['otu_ids'].append(1)

In [41]:
# returns an integer value for the weekly washing frequency 'WFREQ'
freq_df = bbb_meta_df[['SAMPLEID', 'WFREQ']]
#freq_df.head()
#freq_df.shape

***end test section***

# ADD NEW CODE HERE

***section covers the /wfreq/sample route ***

In [42]:
wf = bbb_meta_df.set_index('SAMPLEID')
wf.head()

,EVENT,ETHNICITY,GENDER,AGE,WFREQ,BBTYPE,LOCATION,COUNTRY012,ZIP012,COUNTRY1319,ZIP1319,DOG,CAT,IMPSURFACE013,NPP013,MMAXTEMP013,PFC013,IMPSURFACE1319,NPP1319,MMAXTEMP1319,PFC1319
SAMPLEID,,,,,,,,,,,,,,,,,,,,,
940,BellyButtonsScienceOnline,Caucasian,F,24.0,2.0,I,Beaufort/NC,usa,22306,usa,22306,no,no,8852.0,37.172222,54.5,NaN,1,NaN,33.990002,25.5
941,NaN,Caucasian/Midleastern,F,34.0,1.0,I,Chicago/IL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
943,BellyButtonsScienceOnline,Caucasian,F,49.0,1.0,I,Omaha/NE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
944,BellyButtonsScienceOnline,European,M,44.0,1.0,I,NewHaven/CT,usa,7079,usa,8822,no,yes,NaN,35.816666,16.0,NaN,0,6567.0,32.403332,28.5
945,BellyButtonsScienceOnline,Caucasian,F,48.0,1.0,I,Philidelphia/PA,usa,84404,usa,96025,no,no,NaN,37.783333,4.0,NaN,0,5613.0,33.634445,24.0


In [43]:
samplewf = 'BB_940'

In [44]:
samplewf_ID = int(samplewf.replace("BB_",""))
samplewf_ID

940

In [46]:
wf_new = wf.loc[[samplewf_ID],['WFREQ']]
wf_new

,WFREQ
SAMPLEID,
940,2.0


In [47]:
results2 = wf_new["WFREQ"].item()
results2

2.0

In [48]:
wfreq = freq_df['WFREQ'].tolist()

# section covers the sample / sample route

In [49]:
bbb_samplesDF = pd.read_csv('belly_button_biodiversity_samples.csv')
bbb_samplesDF.head()

,otu_id,BB_940,BB_941,BB_943,BB_944,BB_945,BB_946,BB_947,BB_948,BB_949,BB_950,BB_952,BB_953,BB_954,BB_955,BB_956,BB_958,BB_959,BB_960,BB_961,BB_962,BB_963,BB_964,BB_966,BB_967,BB_968,BB_969,BB_970,BB_971,BB_972,BB_973,BB_974,BB_975,BB_978,BB_1233,BB_1234,BB_1235,BB_1236,BB_1237,BB_1238,BB_1242,BB_1243,BB_1246,BB_1253,BB_1254,BB_1258,BB_1259,BB_1260,BB_1264,BB_1265,...,BB_1513,BB_1514,BB_1515,BB_1516,BB_1517,BB_1518,BB_1519,BB_1521,BB_1524,BB_1526,BB_1527,BB_1530,BB_1531,BB_1532,BB_1533,BB_1534,BB_1535,BB_1536,BB_1537,BB_1539,BB_1540,BB_1541,BB_1542,BB_1543,BB_1544,BB_1545,BB_1546,BB_1547,BB_1548,BB_1549,BB_1550,BB_1551,BB_1552,BB_1553,BB_1554,BB_1555,BB_1556,BB_1557,BB_1558,BB_1561,BB_1562,BB_1563,BB_1564,BB_1572,BB_1573,BB_1574,BB_1576,BB_1577,BB_1581,BB_1601
0,1,0.0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0.0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0.0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0.0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,7,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,0.0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
# enter a random sample value
sample_random = 'BB_1293'

In [54]:
# create a new df object that filters sample_random & otu_id
bbb_samplesDF_filter = bbb_samplesDF.filter(items = [sample_random, 'otu_id'])

In [55]:
# everytime we change sample_random, the data frame should filter to that SAMPLEID
# *****do not copy into app.py*****
bbb_samplesDF_filter.head()

,BB_1293,otu_id
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


In [56]:
# create a new object to sort filtered DF by DESC
bbb_samplesDF_filter_sort = bbb_samplesDF_filter.sort_values(by=sample_random, ascending=False)

In [57]:
# *****do not copy into app.py*****
bbb_samplesDF_filter_sort.head()

,BB_1293,otu_id
3642,128,3643
1166,101,1167
40,89,41
2858,39,2859
1168,39,1169


In [58]:
# create df object to sort > 0
bbb_sampleDF_nan1 = bbb_samplesDF_filter_sort[bbb_samplesDF_filter_sort > 0]
# drops null
bbb_sampleDF_nan2 = bbb_sampleDF_nan1.dropna()

# *****do not copy into app.py*****
bbb_sampleDF_nan2.head()

,BB_1293,otu_id
3642,128.0,3643
1166,101.0,1167
40,89.0,41
2858,39.0,2859
1168,39.0,1169


In [61]:
# this takes the sample_random col or the SAMPLEID col and stores it into a list
sample_val = bbb_sampleDF_nan2[sample_random].tolist()

# *****do not copy into app.py*****
# sample_val

In [64]:
# stores the otu_id col into a list
otu_ids = bbb_sampleDF_nan2['otu_id'].tolist()

# *****do not copy into app.py*****
# otu_ids

In [65]:
# create a dictionary stored in a list
otu_samp_dict = [{'otu_ids': otu_ids, 'sample_values': sample_val}]

In [66]:
otu_samp_dict

[{'otu_ids': [3643,
   1167,
   41,
   2859,
   1169,
   854,
   2539,
   352,
   1977,
   2011,
   728,
   830,
   307,
   2419,
   1208,
   944,
   1189,
   2985,
   1314,
   3450,
   482,
   922,
   296,
   1286,
   2722,
   2964,
   408,
   874,
   1326,
   454,
   2191,
   258,
   2244,
   1870,
   1197,
   1795,
   3636,
   180,
   909,
   84,
   2389,
   2571,
   2782,
   2648,
   1244,
   2883,
   2184,
   2266,
   2666,
   121,
   833,
   841,
   502,
   2167,
   943,
   2461,
   1876,
   92,
   2664,
   99,
   377,
   963,
   1335,
   2662,
   2333,
   1174,
   1232,
   2275,
   1172,
   2412,
   1882,
   1137,
   2264,
   530,
   3541,
   2870,
   158,
   1086,
   233,
   2866,
   2873,
   350,
   2874,
   144,
   2255,
   2558,
   2762,
   165,
   3104,
   670,
   2565,
   1141,
   130,
   2551,
   2073,
   2680,
   1302,
   2820,
   3625,
   189,
   1261,
   1929,
   1283,
   1926,
   3650,
   2594,
   2711,
   1200,
   2378,
   813,
   2375,
   3004,
   1114,
   1204,
   